## Import Libraries

In [1]:
# basic 
import pandas as pd 
import numpy as np 
import klib
import seaborn as sns 
import matplotlib.pyplot as plt  

# model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

# other
import warnings 
warnings.filterwarnings('ignore')

## Load Dataset

In [2]:
# preview dataset
raw = pd.read_excel(r'D:\Project\Test_Shimano\Data.xlsx')
raw.head(15).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
Years,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014
Month,Jan,Jan,Jan,Jan,Jan,Jan,Jan,Jan,Jan,Jan,Feb,Feb,Feb,Feb,Feb
Week,2nd,2nd,2nd,2nd,3rd,3rd,3rd,3rd,4th,4th,1st,1st,1st,2nd,2nd
FindingArea,FG,FG,OTC,FG,FG,FG,FG,FG,FG,FG,FG,FG,FG,FG,FG
Factory,ABC,ABC,ABC,ABC,ABC,ABC,ABC,ABC,HCL,ABC,ABC,ABC,ABC,ABC,ABC
GroupingFactory,ABC,ABC,ABC,ABC,ABC,ABC,ABC,ABC,SUBCON,ABC,ABC,ABC,ABC,ABC,ABC
DepartmentResponsible,Mr. Februari,Mr. Februari,Mr. Maret,Mr. April,Mr. April,Mr. Mei,Mr. Januari,Mr. Januari,Mr. Dafit,Mr. Januari,Mr. Januari,Mr. Januari,Mr. Januari,Mr. Januari,Mr. June
Product,SG,SM,FD,BB,BB,RD,SL,SL,CS,SL,SL,SL,SL,SL,DH
MainModel,SG 3C41,SM-CJ8S20,FD-R2030,BB-ES300,BB-UN26K,RD M310,ST-EF41,ST-EF41,CS-HG201,SL-MT500,SL-M3100,ST-EF65,ST-EF41,ST-EF65,DH-C30003N
Model/Production Code,SG 3C41 (168mm) (LH non-turn) \n235U7010326,SM-CJ8S20 Unit Set GP1\n30015685,FD-R2030 (BRAZED ON) OTC (Packing)\n22B22400037,BB-ES300 113 73\n21V90003056,BB-UN26K LL123 68\n21S1D120356,RD M310 OTC (Packing)\n25W87200237,ST-EF41 (F) 3 SPEED\n26UC2000056,ST-EF41 (F) 3 SPEED\n26UC2000056,CS-HG201 BO\n40050112,SL-MT500-IL\n20L72001056,SL-M3100-2L\n20LJ2000066,ST-EF65-2A (F) 3 SPEED\n26UG2001256,ST-EF41 (F) 3 SPEED\n26UC2000056,ST-EF65-2A (F) 3 SPEED\n26UG2001256,DH-C30003N (Nut Type)\n22AV8001126


In [3]:
print(raw.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Years                  287 non-null    int64         
 1   Month                  287 non-null    object        
 2   Week                   287 non-null    object        
 3   FindingArea            287 non-null    object        
 4   Factory                287 non-null    object        
 5   GroupingFactory        287 non-null    object        
 6   DepartmentResponsible  287 non-null    object        
 7   Product                287 non-null    object        
 8   MainModel              287 non-null    object        
 9   Model/Production Code  287 non-null    object        
 10  QTY                    286 non-null    object        
 11  SamplingChek           287 non-null    int64         
 12  SamplingNC             287 non-null    int64         
 13  NC % 

In [4]:
# melihat statistik dasar dataset
raw.describe()

,Years,SamplingChek,SamplingNC,NC %,DateProduce,TimeProduce
count,287.000000,287.000000,287.000000,287.000000,171,171.000000
mean,2016.020906,59.195122,3.121951,0.188900,2018-09-24 19:22:06.315789568,11.123977
min,2014.000000,1.000000,1.000000,0.001667,2016-12-31 00:00:00,0.050000
25%,2014.000000,20.000000,1.000000,0.033333,2017-11-08 00:00:00,8.310000
50%,2016.000000,25.000000,1.000000,0.050000,2018-12-13 00:00:00,10.310000
75%,2018.000000,50.000000,2.000000,0.200000,2019-05-15 00:00:00,14.235000
max,2018.000000,3000.000000,53.000000,1.000000,2024-04-22 00:00:00,23.560000
std,1.584314,224.852423,5.897947,0.285212,NaN,5.152131


In [5]:
# Menampilkan data duplicate dari dataset
duplicates = raw[raw.duplicated()]
print("Jumlah data duplicate:", duplicates.shape[0])
print(duplicates)

Jumlah data duplicate: 0
Empty DataFrame
Columns: [Years, Month, Week, FindingArea , Factory, GroupingFactory, DepartmentResponsible, Product, MainModel, Model/Production Code, QTY, SamplingChek, SamplingNC, NC %, NCDescription, TypeOfNC, Factor, GroupingFactor, LeaderName, DateProduce, TimeProduce, Shift]
Index: []

[0 rows x 22 columns]


In [ ]:
klib.missing

In [ ]:
# Menampilkan nilai unik dari setiap kolom
def display_unique_values(df):
    unique_values = {}
    for column in df.columns:
        unique_values[column] = df[column].unique()
    return unique_values

unique_values = display_unique_values(raw)

# Menampilkan hasil
for column, values in unique_values.items():
    print(f"Unique values in column '{column}':\n{values}\n")

## Data Distribution

In [ ]:
# Untuk variabel numerik
numeric_columns = raw.select_dtypes(include=['int64', 'float64']).columns
for col in numeric_columns:
    plt.figure(figsize=(10, 6))
    sns.histplot(raw[col], kde=True)
    plt.title(f'Distribusi {col}')
    plt.show()

# Untuk variabel kategorikal
categorical_columns = raw.select_dtypes(include=['object']).columns
for col in categorical_columns:
    plt.figure(figsize=(10, 6))
    raw[col].value_counts().plot(kind='bar')
    plt.title(f'Distribusi {col}')
    plt.xticks(rotation=45)
    plt.show()

## Korelasi

In [ ]:
# mapping untuk mengubah bulan menjadi angka
month_mapping = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
    'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}

# terapkan mapping ke kolom month
raw['Month_Numeric'] = raw['Month'].map(month_mapping)

# drop kolom Month
raw = raw.drop(columns=['Month'], axis=1)

In [ ]:
# Pilih hanya kolom numerik
numeric_columns = raw.select_dtypes(include=['int64', 'float64']).columns

plt.figure(figsize=(12, 10))
sns.heatmap(raw.corr(), annot=True, cmap='coolwarm')
plt.title('Heatmap Korelasi')
plt.show()

## Missing Values

In [ ]:
# check missing values
missing_values = raw.isnull().sum()

# tampilkan missing values
missing_values

In [ ]:
missing_num = 116/287 * 100
missing_num

Susunan dataset dapat terlihat diatas, dimana terdapat **21 kolom** dengan jumlah entri data sebanyak **287 entri**. Untuk kolom seperti `LeaderName`, `DataProduce`, `TimeProduce`, `Shift` yang memiliki data missing sebanyak 116 entri dari 287 atau **>40%** dari total data sehingga untuk handling missing values perlu dilakukan analisa lebih lanjut.

In [ ]:
# menghapus 2 kolom dengan missing values > 40%
columns_to_drop = ['LeaderName', 'DataProduce', 'TimeProduce', 'Shift']
raw = raw.drop(columns=columns_to_drop)
# mengisi missing values dengan median
